In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
import time

import random

import torch
import torch.nn as nn
import torch.nn.functional as F


import torch.optim as optim

import copy
import matplotlib.pyplot as plt
import math

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'



In [ ]:
data = pd.read_csv('station_153211-2022-09_2023-01.csv')


In [ ]:
# Adding / removing columns.
data['DateTime'] = pd.to_datetime(data['created_at'], infer_datetime_format=True)
data.set_index('DateTime', inplace=True)
data['day_of_week'] = data.index.dayofweek
data['month'] = data.index.month
data['is_weekend'] = (data.index.dayofweek >= 5).astype(int)
data['hour_bin'] = data.index.hour // 4 
holidays = pd.to_datetime(['2022-12-31', '2022-12-23'])
data['is_holiday'] = data.index.isin(holidays).astype(int)
data.drop('created_at', axis=1, inplace=True)
data.drop('station_id', axis=1, inplace=True)
data.drop('available_count', axis=1, inplace=True)
data.drop('outlet_count', axis=1, inplace=True)

In [ ]:
data = data[~data.index.duplicated(keep='first')]

start_date = data.index[5]
end_date = data.index[-5]
datetime_index = pd.date_range(start=start_date, end=end_date, freq='5T')

data_r = pd.DataFrame(index=datetime_index, columns=data.columns)

k = 10

for target_time in datetime_index:

    data_c = copy.copy(data)
    data_c['time_difference'] = abs(data_c.index - target_time)
    data_c_sorted = data_c.sort_values(by='time_difference')
    k_closest_rows = data_c_sorted.head(k)
    k_closest_rows= k_closest_rows.drop(columns=['time_difference'])
    new_col = k_closest_rows.mean(axis=0)
    data_r.loc[target_time] = new_col

In [ ]:

scaler = MinMaxScaler()

df_normalized = pd.DataFrame(scaler.fit_transform(data_r),
                             index=data_r.index,
                             columns=data_r.columns)

print(df_normalized['is_holiday'].value_counts())
(data['is_holiday'].value_counts())


In [ ]:
all_labels = df_normalized['occupied_count']
print(all_labels)

In [ ]:
series_length = 36327
random_numbers = pd.Series(np.random.rand(series_length))

random_numbers.index = all_labels.index

print(random_numbers)


In [ ]:
# Convert pandas Series to PyTorch tensors
random_numbers_tensor = torch.tensor(random_numbers.values, dtype=torch.float32)
all_labels_tensor = torch.tensor(all_labels.values, dtype=torch.float32)

# Calculate mean squared error
mse = F.mse_loss(random_numbers_tensor, all_labels_tensor)

print("Mean Squared Error:", mse.item())


In [ ]:
mean_series = torch.tensor(pd.Series(all_labels.mean(), index=range(series_length)).values, dtype=torch.float32)

# Calculate mean squared error
mse = F.mse_loss(mean_series, all_labels_tensor)

print("Mean Squared Error:", mse.item())


